In [2]:
import numpy as np
import itertools
import pandas as pd 
from itertools import product
import random
from six.moves import cPickle as pickle

import gc


# data_root = "C:\\Users\\Tigran PC\\Desktop\\MicrobiomicsData\\data\\" # notebook
# data_root = "/Users/tigran/Desktop/sbv/data/" # imac
data_root = "C:\\Users\\Administrator\\Repos\\Microbiomics\\data\\" # server

In [4]:
def make_sub_seq_bank(initial_string, sub_seq_len):
    return [''.join(tup) for tup in  list(set(product(set(initial_string), repeat = sub_seq_len)))]

    
def convert_read_to_numb(dataframe, sub_str_len):
    ss_bank =  make_sub_seq_bank("ATCG", sub_str_len)
    data_list = []
    for seq in dataframe.values:
        sub_seq_count = []
        for ss in ss_bank:
            sub_seq_count.append(seq[0].count(ss))
        sub_seq_count.append(seq[1])      
        data_list.append(sub_seq_count)
    return pd.DataFrame(data_list, columns=ss_bank.append("lable"))    


In [5]:
df1 = pd.read_pickle(data_root + "DNA_data1.pickle")

In [6]:
df2 = pd.read_pickle(data_root + "DNA_data2.pickle")

In [50]:
# print(df1.shape)
# print(df2.shape)
# df = pd.concat([df1, df2]) 
# del df1, df2
# df.head()

In [51]:
print(df.shape)
df.head()

(78731623, 2)


,0,lable
0,TTACGACACCTGGTCGACGGTGTACAACCAGCTCGAGGGCACTTGG...,1768
1,GAACGGCCGCAGCTCGTCGATCACCCCGGTCAGCGCCCGCGTCTCC...,1751294
2,AGCCTATATTAAATTGCTACCGCCTGAAAAACGAGGAGCGGAGAAC...,189834
3,TGGTATTTCTCTGTATCAAAATTGGCGTTTTGATAATAATACGGGA...,1954172
4,GTAAAGGTCAGTACGGCCACGTGGTCTTCACGCTCGAACCATTGCC...,741091


In [84]:
step = 30000000

In [85]:
df_s = pd.DataFrame(df.values[20000000:step, :])

In [89]:
df_s.head()

,0,1
0,TCCCATTGGGGTTCCACGAACAAGGGTTTCTCTTCATCAACGGCAG...,1308
1,CCCGCCGACGGATGCGTCGGCCTGGCTGGCCAGGCCCAGTGACAGG...,1768
2,AACTCGGCGCCCTGGCCCCGCTGATCGACTGGACAAGCCAGATGGC...,56730
3,CTACGCCAATGCAGCAAAGCTTATGGCTGCAGGTGCAAGTATGGTT...,28108
4,CCGCGACCCGCGCGTGGTGCAGCAACTGGAAGCCGCCAAGCGCGAA...,864051


### Convert to numbers

In [90]:
gc.collect()
df_s.shape

(10000000, 2)

In [ ]:
%time df_s =  convert_read_to_numb(df_s, 4) 

In [80]:
df_s.head(10)

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,256
0,0,0,0,0,1,0,0,2,0,1,...,1,0,0,0,1,0,1,0,0,53408
1,0,0,0,1,0,1,0,2,3,3,...,2,0,1,2,1,0,0,0,0,104102
2,1,1,1,0,0,0,0,1,1,0,...,0,0,0,2,1,2,1,0,0,412690
3,0,2,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,1,0,0,225848
4,2,1,1,0,0,0,1,1,1,0,...,1,0,0,3,1,1,0,1,0,797277
5,0,0,0,0,1,1,0,0,0,0,...,0,0,1,2,0,0,0,0,0,1355477
6,0,0,0,1,0,0,1,1,1,1,...,0,0,0,0,2,0,0,0,0,722472
7,0,0,0,0,0,0,0,0,5,1,...,8,0,0,0,4,0,0,0,0,1458426
8,0,1,0,0,0,0,0,1,1,3,...,0,0,0,0,1,0,0,0,0,260552
9,0,0,0,1,2,0,0,0,1,0,...,2,1,0,0,2,0,1,1,0,1379910


In [81]:
columns = list(df_s.columns.values)
print(type(columns))
df_s = df_s.sample(frac=1).reset_index(drop=True)
print(df_s.shape)
df_s = df_s.drop_duplicates(subset=columns[:-1])
print(df_s.shape)

<class 'list'>
(10000000, 257)
(9999904, 257)


In [82]:
df_s.tail()

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,256
9999995,2,1,1,0,0,0,1,1,0,3,...,1,1,0,0,2,0,1,1,0,317
9999996,0,0,0,0,0,0,1,1,0,0,...,0,0,0,6,1,0,1,0,0,1714373
9999997,0,0,0,0,0,0,0,0,2,0,...,0,0,1,0,1,0,1,0,0,169283
9999998,0,1,0,0,0,0,0,0,4,0,...,1,0,0,2,0,0,0,0,0,196162
9999999,0,0,1,0,1,1,1,0,2,0,...,0,0,0,0,0,0,1,0,1,587753


In [83]:
pickle_file = data_root + "F_Bank_2.pickle"

try:   
    f = open(pickle_file, 'wb')
    pickle.dump(df_s, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise